In [ ]:
#downloading the required packages
%pip install pandas
%pip install pandas_ta
%pip install numpy
%pip install matplotlib
%pip install statsmodels
%pip install pandas_datareader
%pip install datetime
%pip install yfinance
%pip install sklearn
%pip install PyPortfolioOpt

In [3]:
# from statsmodels.regression.rolling import RollingOLS
# import pandas_datareader.data as web
# import matplotlib.pyplot as plt
# import statsmodels.api as sm
# import pandas as pd
# import numpy as np
# import yfinance as yf
# import pandas_ta
# import warnings
# warnings.filterwarnings('ignore')


# sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

# symbols_list = sp500['Symbol'].unique().tolist()

# symbols_list

# end_date = '2025-01-21'
# start_date = pd.to_datetime(end_date) - pd.DateOffset(365*10)
# df = yf.download(
#     tickers=symbols_list,
#     start=start_date,
#     end=end_date,
#     auto_adjust=False,   # <-- keep Adj Close
#     group_by='column'    # default, but explicit is nice
# ).stack()
# df.index.names = ['date', 'ticker']
# df.columns = df.columns.str.lower()
# df


[*********************100%***********************]  503 of 503 completed


Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-01-26 A        35.902451   39.150002   39.169998   38.470001   38.790001   
           AAPL     25.097633   28.275000   28.590000   28.200001   28.434999   
           ABBV     40.729500   62.830002   63.040001   61.680000   62.259998   
           ABT      36.159981   44.150002   44.200001   43.419998   43.740002   
           ACGL     18.932390   19.910000   19.913334   19.623333   19.836666   
...                       ...         ...         ...         ...         ...   
2025-01-17 XYZ      86.959999   86.959999   88.150002   86.099998   87.000000   
           YUM     124.121735  125.320000  127.349998  125.190002  126.910004   
           ZBH     109.042999  109.559998  109.970001  108.470001  108.989998   
           ZBRA    405.709991  405.709991  407.290009  402.290009  406.040009   
           ZTS     164.284439  165.889999  169.669998  165.809998  168.899994   

Price                   volume  
date       ticker               
2015-01-26 A         1510900.0  
           AAPL    222460000.0  
           ABBV      8181100.0  
           ABT       4376200.0  
           ACGL      1662000.0  
...                        ...  
2025-01-17 XYZ       5797000.0  
           YUM       1604900.0  
           ZBH       1097100.0  
           ZBRA       270600.0  
           ZTS       4657200.0  

[1226482 rows x 6 columns]

In [15]:
# # calculate the Garman - Klass Volatility
# df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low']))**2) / 2 - (2*np.log(2) - 1) * (np.log(df['adj close']) - np.log(df['open']))**2


# # calculate rsi
# df['rsi'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

# df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:,0])

# df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:,1])

# df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:,2])

# def compute_atr(stock_data):
#     atr = pandas_ta.atr(high = stock_data['high'],
#                         low = stock_data['low'],
#                         close = stock_data['close'])
#     return atr.sub(atr.mean()).div(atr.std())   #normalizing it

# df['atr'] = df.groupby(level=1, group_keys = False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close= close, length = 20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())  #normalizing it

df['macd'] = df.groupby(level = 1, group_keys = False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df
    

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-01-26 A        35.902451   39.150002   39.169998   38.470001   38.790001   
           AAPL     25.097633   28.275000   28.590000   28.200001   28.434999   
           ABBV     40.729500   62.830002   63.040001   61.680000   62.259998   
           ABT      36.159981   44.150002   44.200001   43.419998   43.740002   
           ACGL     18.932390   19.910000   19.913334   19.623333   19.836666   
...                       ...         ...         ...         ...         ...   
2025-01-17 XYZ      86.959999   86.959999   88.150002   86.099998   87.000000   
           YUM     124.121735  125.320000  127.349998  125.190002  126.910004   
           ZBH     109.042999  109.559998  109.970001  108.470001  108.989998   
           ZBRA    405.709991  405.709991  407.290009  402.290009  406.040009   
           ZTS     164.284439  165.889999  169.669998  165.809998  168.899994   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-01-26 A         1510900.0         -0.002149        NaN       NaN   
           AAPL    222460000.0         -0.005927        NaN       NaN   
           ABBV      8181100.0         -0.069329        NaN       NaN   
           ABT       4376200.0         -0.013832        NaN       NaN   
           ACGL      1662000.0         -0.000733        NaN       NaN   
...                        ...               ...        ...       ...   
2025-01-17 XYZ       5797000.0          0.000277  50.023712  4.413906   
           YUM       1604900.0         -0.000044  36.345901  4.803844   
           ZBH       1097100.0          0.000094  57.332150  4.630622   
           ZBRA       270600.0          0.000076  56.778174  5.939196   
           ZTS       4657200.0         -0.000032  44.585695  5.080432   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2015-01-26 A            NaN       NaN       NaN       NaN      54.245013  
           AAPL         NaN       NaN       NaN       NaN    5583.219518  
           ABBV         NaN       NaN       NaN       NaN     333.212111  
           ABT          NaN       NaN       NaN       NaN     158.243308  
           ACGL         NaN       NaN       NaN       NaN      31.465633  
...                     ...       ...       ...       ...            ...  
2025-01-17 XYZ     4.481450  4.548995 -0.165614 -0.334718     504.107115  
           YUM     4.866858  4.929872  0.877267 -2.305157     199.202972  
           ZBH     4.664857  4.699093 -0.360653 -0.053789     119.631074  
           ZBRA    5.973450  6.007705  0.482055  0.026113     109.785124  
           ZTS     5.101197  5.121962  0.706787 -0.933300     765.105490  

[1226482 rows x 14 columns]

In [ ]:
# last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume','open','high', 'low','close']]

# df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume')

df.unstack()[last_cols].resample('M').last().stack('ticker')

Price               adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2015-01-31 A        34.636921         -0.003207        NaN       NaN   
           AAPL     25.998579         -0.006148        NaN       NaN   
           ABBV     39.121838         -0.081549        NaN       NaN   
           ABT      36.659573         -0.017452        NaN       NaN   
           ACGL     18.374529         -0.001528        NaN       NaN   
...                       ...               ...        ...       ...   
2025-01-31 XYZ      86.959999          0.000277  50.023712  4.413906   
           YUM     124.121735         -0.000044  36.345901  4.803844   
           ZBH     109.042999          0.000094  57.332150  4.630622   
           ZBRA    405.709991          0.000076  56.778174  5.939196   
           ZTS     164.284439         -0.000032  44.585695  5.080432   

Price                bb_mid   bb_high       atr      macd  
date       ticker                                          
2015-01-31 A            NaN       NaN       NaN       NaN  
           AAPL         NaN       NaN       NaN       NaN  
           ABBV         NaN       NaN       NaN       NaN  
           ABT          NaN       NaN       NaN       NaN  
           ACGL         NaN       NaN       NaN       NaN  
...                     ...       ...       ...       ...  
2025-01-31 XYZ     4.481450  4.548995 -0.165614 -0.334718  
           YUM     4.866858  4.929872  0.877267 -2.305157  
           ZBH     4.664857  4.699093 -0.360653 -0.053789  
           ZBRA    5.973450  6.007705  0.482055  0.026113  
           ZTS     5.101197  5.121962  0.706787 -0.933300  

[59084 rows x 8 columns]